<a href="https://colab.research.google.com/github/nalewkoz/neural-bifurcations/blob/main/Hopfield_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Noiseless ($T=0$) Hopfield network

Let's simulate the Hopfield network with random patterns. The control parameter we focus here on is the ratio of the number of stored patterns to the number of neurons $\alpha = P/N$. According to the mean-field theory, there is a phase transition at $\alpha_c \approx 0.14$. Let's check if we can observe it here.

In [ ]:
# @title 
# @markdown Run this cell to import packages and load helper functions

# Standard imports

import numpy as np
import matplotlib.pyplot as plt
import time
import torch
from matplotlib import rc
rc('font',**{'size':20})

precision = torch.double

## Check if a GPU available
if torch.cuda.is_available():
    print("GPU available: "+torch.cuda.get_device_name())
    dev = "cuda:0"
else:
    print("No GPU available... Using CPU.")
    dev = "cpu:0"

device = torch.device(dev)
torch.cuda.empty_cache()


def draw_evolution(X, mall, Nshow = -1, Tshow = -1, Pshow = -1):
#fig = plt.figure(figsize=(20,8))
    fig, ax = plt.subplots(1,2, figsize=(18,8))
    
    Z = X[0:Nshow,0:Tshow]
    ax[0].imshow(Z)
    ax[0].set_xlabel('Time')

    mall = mall[:Pshow,:Tshow]
    ax[1].plot(range(mall.size()[1]),mall.T)
    ax[1].set_xlabel('Time')
    ax[1].set_ylabel('m')

def generate_patterns(N, P):
    # Generate P random patterns of size N
    patterns = 2*torch.randint(2,(N,P),device=device).type(precision)-1
    return patterns

def generate_J_mask(patterns, mask):
    print("Generate J... ", end='')
    N       = patterns.size()[0]
    Jfull   = torch.matmul(patterns, patterns.T)
    J       = Jfull*mask

    Jdiag   = J.diagonal()
    Jdiag   *= 0
    print("Done!")
    return J

def generate_J(patterns):
    print("Generate J... ", end='')
    N       = patterns.size()[0]
    J       = torch.matmul(patterns, patterns.T) 

    Jdiag   = J.diagonal()
    Jdiag   *= 0
    print("Done!")
    return J

def evolve_nn(J, x0, Nepoch=10):  
    print("Simulate... ", end='')
    N       = J.size()[0]
    X = torch.zeros( (N, Nepoch+1), device=device, dtype=precision )
    X[:,0] = x0
    for i in range(Nepoch):
        # Asynchronous update (much slower here, but should be closer the theoretical predictions)
        X[:,i+1] = X[:,i]
        for j in range(N):
            X[j,i+1] =  2*(torch.matmul(J[j,:], X[:,i+1]) > 0 ) - 1
        # Synchronous update:
        #X[:,i+1] =  2*(torch.matmul(J, X[:,i]) > 0 ) - 1
    print("Done!")
    return X

def calc_corrs(patterns, ind=0):
    return torch.matmul(patterns.T, patterns[:,0])/N

In [ ]:
## Parameters
N       = int(1e3)
Nepoch  = int(1e2)
alpha   = ...

initErr = 0.1

Pshow   = 10

start = time.time()
## Generate patterns and weights
P           = int(alpha*N)
patterns    = generate_patterns(N,P)
J           = generate_J(patterns[:,:P])

x0 = torch.clone(patterns[:,0])                 # Initial condition at 0th pattern...
x0[0:int(initErr*N)] = -x0[0:int(initErr*N)]    # ...but we introduce some error (flip some bits)

## Simulate
X    = evolve_nn(J, x0 , Nepoch=Nepoch)

## Analyze
mall = torch.matmul(patterns[:,:Pshow].T, X)/N
X    = X.cpu()#.numpy()
mall = mall.cpu()
stop = time.time()

print(f"Time on {dev} (torch): "+str(stop-start))

draw_evolution(X, mall, Nshow=Nepoch+1, Pshow=Pshow)

Modify the code to incorporate diluted asymmetric connections. Note that the code to generate diluted synaptic weights is provided in the function _generate_J_mask_. You still have to generate the mask, which should contain many 0's (with probability $1 - K/N$) and some 1's (with probability $K/N$). $K$ should be much smaller than $N$, and in this case we redefine $\alpha$ as $\alpha = P/K$.

How is the behavior of the system different in this case?

See the original paper for details:

_[Derrida, B., Gardner, E., & Zippelius, A. (1987). An exactly solvable asymmetric neural network model. EPL (Europhysics Letters), 4(2), 167.
](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.130.1431&rep=rep1&type=pdf)_